In [254]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
%matplotlib inline

---

In [255]:
#DBM data below

In [256]:
DBM = pd.DataFrame()
for file in glob.glob("DBM*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    DBM = DBM.append(df, ignore_index=True)

In [257]:
DBM = DBM.loc[:, ["Month", "Advertiser", "Public Inventory", "Impressions", 
                  "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency"]]

In [258]:
DBM.columns = ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency"]

In [259]:
DBM["DSP"] = "DBM"
DBM["Type"] = "General"

---

In [260]:
#YouTube/TrueView data below

In [261]:
YT = pd.DataFrame()
for file in glob.glob("YT*.csv"):
    df = pd.read_csv(file)
    df = df[df["Advertiser Currency"].isnull() == False]
    YT = YT.append(df, ignore_index=True)

In [262]:
YT["Public"] = "N/A"
YT["DSP"] = "DBM"
YT["Type"] = "TrueView"

In [263]:
YT = YT.loc[:, ["Month", "Advertiser", "Public","Impressions", 
                "Clicks", "Revenue (Adv Currency)", "Media Cost (Advertiser Currency)", "Advertiser Currency", "DSP", "Type"]]

In [264]:
YT.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks',
       'Spend', 'Media', 'Currency', 'DSP', 'Type']

---

In [265]:
#Videology data below
#please note the data in the template the time frame please select the specific time rather than 'last month'

In [266]:
videology = pd.DataFrame()
for file in glob.glob("Videology*.csv"):
    df = pd.read_csv(file)
    df = df[df["Placement ID"].isnull() == False]
    videology = videology.append(df, ignore_index=True)

In [267]:
videology.columns = ["Advertiser", "Placement", "PlacementName", "Date", "Impressions", "Spend", "Clicks"]

In [268]:
list1=[]
for month in videology.Date:
    g = month[-4:] + "/" + "{0:0=2d}".format(time.strptime(month[:3], "%b").tm_mon)
    list1.append(g)

In [269]:
#repmoving the thousand separator 
videology.Impressions = videology.Impressions.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Clicks = videology.Clicks.apply(lambda x: float(x.split()[0].replace(',', '')))
videology.Spend = videology.Spend.apply(lambda x: float(x.split()[0].replace(',', '')))

In [270]:
videology["Currency"] = "GBP"
videology["DSP"] = "Videology"
videology["Type"] = "General"
videology.loc[(videology.PlacementName.str.contains("Channel 4")) | 
              (videology.PlacementName.str.contains("C4")), "Type"] = "Channel 4"
videology["Media"] = videology.Spend * 0.7
videology.loc[videology.Type == "Channel 4", "Media"] = videology.Spend * 0.8
videology["Public"] = "N/A"
videology["Month"] = list1

In [271]:
videology = videology.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend',
       'Media', 'Currency', 'DSP', 'Type']]

---

In [272]:
#TheTradeDesk data below

In [273]:
TTD = pd.DataFrame()
for file in glob.glob("TTD*.tsv"):
    df = pd.read_csv(file)
    TTD = TTD.append(df, ignore_index=True)
TTD.columns = ["all"]
TTD = pd.DataFrame(TTD["all"].str.split("\t").tolist(), 
             columns=["Date", "Advertiser","Currency", "PMP", "Impressions", "Clicks", "Spend", "Media"])

In [274]:
TTD["Month"] = TTD.Date.str.split("/").str[2] + "/" + TTD.Date.str.split("/").str[1]
TTD["Public"] = "Yes"
TTD.loc[TTD.PMP.str.contains("PMP"), "Public"] = "No"
TTD["DSP"] = "TTD"
TTD["Type"] = "General"

In [275]:
TTD = TTD.loc[:, ["Month", "Advertiser", "Public", "Impressions", "Clicks", "Spend", "Media", "Currency", "DSP", "Type"]]

In [276]:
TTD.Impressions = TTD.Impressions.astype("float64")
TTD.Clicks = TTD.Clicks.astype("float64")
TTD.Spend = TTD.Spend.astype("float64")
TTD.Media = TTD.Media.astype("float64")

In [277]:
#below is fixed for 2016 TTD data 

In [278]:
TTD_old = pd.read_csv("TTD_0116_1216.csv")

In [279]:
TTD_old["Month"] = TTD_old.Date.str.split("/").str[2] + "/" + TTD_old.Date.str.split("/").str[1]
TTD_old["Public"] = "Yes"
TTD_old.loc[TTD_old["Inventory Contract"] == "PMP", "Public"] = "No"
TTD_old["DSP"] = "TTD"
TTD_old["Type"] = "General"

In [280]:
TTD_old = TTD_old.loc[:, ["Month", "Advertiser", "Public", "Impressions", 
                          "Clicks", "Advertiser Cost (Adv Currency)", "Media Cost (Adv Currency)", 
                          "Advertiser Currency Code", "DSP", "Type"]]

In [281]:
TTD_old.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks',
       'Spend', 'Media','Currency', 'DSP', 'Type']

In [282]:
TTD = TTD.append(TTD_old, ignore_index=True)

---

In [283]:
#RocketFuel data below

In [284]:
RF = pd.DataFrame()
for file in glob.glob("RF*.csv"):
    df = pd.read_csv(file, header=1)
    RF = RF.append(df, ignore_index=True)
RF.Spend = RF.Spend.str.replace("GBP ", "")
RF["Media Cost"] = RF["Media Cost"].str.replace("GBP ", "")

In [285]:
RF["Public"] = "Yes"
RF.loc[RF["Exchange Deal ID"].isnull() == False, "Public"] = "No"
RF["Month"] = RF["Year and Month"].str.split("-").str[0] + "/" + RF["Year and Month"].str.split("-").str[1]
RF["DSP"] = "RocketFuel"
RF["Type"] = "General"
RF["Currency"] = "GBP"

In [286]:
RF = RF.loc[:, ["Month", "Advertiser", "Public", "Imps", "Clicks", "Spend", "Media Cost", "Currency", "DSP", "Type"]]

In [287]:
RF.columns = ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend', 'Media', 'Currency', 'DSP', 'Type']

In [288]:
RF.Spend = RF.Spend.astype("float64")
RF.Media = RF.Media.astype("float64")

In [289]:
RF

,Month,Advertiser,Public,Impressions,Clicks,Spend,Media,Currency,DSP,Type
0,2017/03,Oracle_UK_Zenith International,Yes,501277,113,1580.351490,354.304629,GBP,RocketFuel,General
1,2017/01,SIA_UK_Zenith,Yes,10036103,1619,5743.274096,5228.897753,GBP,RocketFuel,General
2,2017/02,SIA_UK_Zenith,Yes,12439864,1619,6834.760327,6221.848560,GBP,RocketFuel,General
3,2017/03,SIA_UK_Zenith,Yes,7411795,1055,3989.397817,3632.332180,GBP,RocketFuel,General
4,2017/04,Oracle_UK_Zenith International,Yes,6520125,3692,21566.779405,5738.533406,GBP,RocketFuel,General
5,2017/04,SIA_UK_Zenith,Yes,820462,147,457.593990,416.580948,GBP,RocketFuel,General
6,2017/05,Oracle_UK_Zenith International,Yes,9632442,6011,30975.331033,9884.922543,GBP,RocketFuel,General
7,2017/05,SIA_UK_Zenith,Yes,1419037,228,712.110331,648.336431,GBP,RocketFuel,General


---

In [290]:
AAP_old = pd.read_csv("Amazon_0116_0317.csv")

In [291]:
AAP_old = AAP_old.head(147)

In [292]:
AAP_old.date = AAP_old.date.str.split(" ").str[0]
AAP_old.date = AAP_old.date.str.split("/").str[2] + "/" + AAP_old.date.str.split("/").str[1]

In [293]:
AAP_old.columns = ['Month', 'Advertiser', 'advertiser_id', 'Impressions', 'Clicks',
       'Media', 'net cost', 'agency markup', 'Spend']

In [294]:
AAP = pd.DataFrame()
for file in glob.glob("AAP*.csv"):
    df = pd.read_csv(file, usecols=["Date", "Advertiser", "Impressions", "Clickthroughs", 
                                    "Total Cost", "Supply Cost", "Order Currency"])
    AAP = AAP.append(df, ignore_index=True) 

In [295]:
AAP = AAP.drop(AAP.index[AAP["Total Cost"] == 0]).reset_index(drop=True)

In [296]:
list2=[]
for month in AAP.Date:
    g = month[-4:] + "/" + "{0:0=2d}".format(time.strptime(month[:3], "%b").tm_mon)
    list2.append(g)

In [297]:
AAP.Date = list2
AAP["Public"] = "N/A"
AAP["DSP"] = "AAP"
AAP["Type"] = "General"

In [298]:
AAP.columns = ['Month', 'Advertiser', 'Currency', 'Media', 'Spend','Impressions', 'Clicks', 'Public', 'DSP', 'Type']

In [299]:
AAP = AAP.loc[:, ['Month', 'Advertiser', 'Public', 'Impressions', 'Clicks', 'Spend', 'Media', 'Currency', 'DSP', 'Type']]

---

In [300]:
#Data from all DSPs

In [301]:
data = DBM.append([TTD, YT, videology, RF, AAP], ignore_index=True)

In [302]:
np.unique(data.Currency)
USDtoGBP = 0.77
EURtoGBP = 0.84

In [303]:
data["cSpend"] = data.Spend 
data.loc[data.Currency == "USD","cSpend"] = data.Spend * USDtoGBP
data.loc[data.Currency == "EUR","cSpend"] = data.Spend * EURtoGBP

---

In [304]:
#file to ZO folder on desktop

In [305]:
data.sort_values("Month", ascending=True).to_csv("D:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend.csv", index=False)

In [309]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], 
                 aggfunc=np.sum, margins=True).round().to_csv("D:\\Users\\ylin\\Desktop\\ZO Spend\\"+"ZO spend pivot.csv")

In [308]:
data.pivot_table(values="cSpend", index="Month", columns=["DSP", "Type", "Public"], aggfunc=np.sum, margins=True).round()

DSP          AAP         DBM                        RocketFuel       TTD  \
Type     General     General               TrueView    General   General   
Public       N/A          No         Yes        N/A        Yes        No   
Month                                                                      
2016/01      NaN    462491.0   2015535.0   126240.0        NaN       NaN   
2016/02      NaN    599799.0   2321375.0   180716.0        NaN       NaN   
2016/03      NaN   1042068.0   2574432.0   279131.0        NaN       NaN   
2016/04      NaN    525837.0   2141692.0   371005.0        NaN       NaN   
2016/05      NaN    537853.0   1718345.0   153631.0        NaN       NaN   
2016/06      NaN    601384.0   1900652.0   286835.0        NaN       NaN   
2016/07      NaN    470456.0   1444104.0   225570.0        NaN   19428.0   
2016/08      NaN    685058.0   1577548.0   138528.0        NaN   29773.0   
2016/09      NaN    843696.0   1788445.0   172688.0        NaN   37963.0   
2016/10      NaN    784882.0   2080459.0   380387.0        NaN       3.0   
2016/11      NaN    987119.0   1905395.0   350509.0        NaN   23429.0   
2016/12      NaN   1018084.0   1783496.0   174375.0        NaN   91615.0   
2017/01      NaN    442769.0   1138666.0   201930.0     5743.0   17489.0   
2017/02      NaN    419031.0    947721.0   175352.0     6835.0   16691.0   
2017/03      NaN    596981.0   1165207.0   139000.0     5570.0   33715.0   
2017/04  25278.0    593276.0   1188412.0   132948.0    22024.0   23500.0   
2017/05      NaN    298205.0    926253.0    63504.0    31687.0   37394.0   
All      25278.0  10908988.0  28617737.0  3552351.0    71860.0  330999.0   

DSP                 Videology                    All  
Type                Channel 4    General              
Public         Yes        N/A        N/A              
Month                                                 
2016/01        0.0        NaN        NaN   2604266.0  
2016/02        0.0        NaN        NaN   3101890.0  
2016/03        NaN        NaN        NaN   3895631.0  
2016/04     2615.0        NaN        NaN   3041149.0  
2016/05    19819.0   121445.0   341517.0   2892609.0  
2016/06    21837.0   163329.0   353411.0   3327449.0  
2016/07    60549.0   104554.0   162814.0   2487475.0  
2016/08   163617.0   215635.0   113612.0   2923772.0  
2016/09    71513.0   302641.0   161261.0   3378207.0  
2016/10    45319.0   831893.0   227547.0   4350490.0  
2016/11    98703.0   841687.0   145597.0   4352439.0  
2016/12    74246.0   480049.0    31477.0   3653341.0  
2017/01    95751.0   264591.0   212558.0   2379496.0  
2017/02   131737.0   518240.0   228212.0   2443819.0  
2017/03   266813.0   433708.0   237404.0   2878399.0  
2017/04   234321.0   518357.0   147026.0   2885142.0  
2017/05   237539.0   483277.0    97798.0   2175656.0  
All      1524380.0  5279405.0  2460233.0  52771231.0